# Play with get_seq_from_index_balance



##### massup_batch is called 4 times inside get_seq_from_index_balance.

One time if: good pedestrian numbers between 256 and 512

One time elif: too many people in current scene > 512, split the scene into two batches

One time if: too few people < 256, accumulate multiple framedata into a batch

At the end, one time

"if last_frame < data_index.shape[1] - 1 and setname == 'test' and batch_pednum > 1:"

In [15]:
import os
import pickle
import random
import time
import numpy as np
import torch

In [16]:
DATASET_NAME_TO_NUM = {
    'eth': 0,
    'hotel': 1,
    'zara1': 2,
    'zara2': 3,
    'univ': 4
}

class Object(object):
    pass

args = Object()
args.dataset = 'eth5'
args.test_set = 'eth'
args.save_dir = './output/' + args.test_set + "/"
args.seq_length = 20
args.obs_length = 20
args.neighbor_thred = 10
args.batch_size = 8
args.batch_around_ped = 256
if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)

self = Object()
self.args = args

setname = 'train'

In [17]:
if self.args.dataset == 'eth5':

    self.data_dirs = ['../data/eth/univ', '../data/eth/hotel',
                      '../data/ucy/zara/zara01', '../data/ucy/zara/zara02',
                      '../data/ucy/univ/students001', '../data/ucy/univ/students003',
                      '../data/ucy/univ/uni_examples', '../data/ucy/zara/zara03']

    # Data directory where the pre-processed pickle file resides
    self.data_dir = './data'
    skip = [6, 10, 10, 10, 10, 10, 10, 10]

    train_set = [i for i in range(len(self.data_dirs))]

    assert args.test_set in DATASET_NAME_TO_NUM.keys(), 'Unsupported dataset {}'.format(args.test_set)

    args.test_set = DATASET_NAME_TO_NUM[args.test_set]

    if args.test_set == 4 or args.test_set == 5:
        self.test_set = [4, 5]
    else:
        self.test_set = [self.args.test_set]

    for x in self.test_set:
        train_set.remove(x)

    self.train_dir = [self.data_dirs[x] for x in train_set]
    self.test_dir = [self.data_dirs[x] for x in self.test_set]
    self.trainskip = [skip[x] for x in train_set]
    self.testskip = [skip[x] for x in self.test_set]
else:
    raise NotImplementedError
    
self.train_data_file = os.path.join(self.args.save_dir, "train_trajectories.cpkl")
self.test_data_file = os.path.join(self.args.save_dir, "test_trajectories.cpkl")
self.train_batch_cache = os.path.join(self.args.save_dir, "train_batch_cache.cpkl")
self.test_batch_cache = os.path.join(self.args.save_dir, "test_batch_cache.cpkl")

if setname == 'train':
    data_dirs = self.train_dir
    data_file = self.train_data_file
else:
    data_dirs = self.test_dir
    data_file = self.test_data_file


def load_dict(data_file):
    f = open(data_file, 'rb')
    raw_data = pickle.load(f)
    f.close()
    frameped_dict = raw_data[0]
    pedtraject_dict = raw_data[1]
    return frameped_dict, pedtraject_dict

In [20]:
def get_data_index(self, data_dict, setname, ifshuffle=True):
    '''
    Get the dataset sampling index.
    '''
    set_id = []
    frame_id_in_set = []
    total_frame = 0
    for seti, dict in enumerate(data_dict):
        frames = sorted(dict)
        # TODO Why are we subtracting 20 frames and not 20 timesteps?!
        #?! maxframe = max(frames) - self.args.seq_length*(frames[1] -
        # frames[0])
        maxframe = max(frames) - self.args.seq_length
        frames = [x for x in frames if not x > maxframe]
        total_frame += len(frames)
        set_id.extend(list(seti for i in range(len(frames))))
        frame_id_in_set.extend(list(frames[i] for i in range(len(frames))))

    all_frame_id_list = list(i for i in range(total_frame))

    data_index = np.concatenate((np.array([frame_id_in_set], dtype=int), np.array([set_id], dtype=int),
                                 np.array([all_frame_id_list], dtype=int)), 0)
    if ifshuffle:
        random.Random().shuffle(all_frame_id_list)
    data_index = data_index[:, all_frame_id_list]

    # to make full use of the data, add the beginning at the end
    if setname == 'train':
        data_index = np.append(data_index, data_index[:, :self.args.batch_size], 1)
    return data_index

In [31]:
def find_trajectory_fragment(trajectory, startframe, seq_length, skip):
    '''
    Query the trajectory fragment based on the index. Replace with 0 if data does not exist.
    '''
    return_trajec = np.zeros((seq_length, 3))
    endframe = startframe + (seq_length) * skip
    start_n = np.where(trajectory[:, 0] == startframe)
    end_n = np.where(trajectory[:, 0] == endframe)
    iffull = False
    ifexsitobs = False

    if start_n[0].shape[0] == 0 and end_n[0].shape[0] != 0:
        start_n = 0
        end_n = end_n[0][0]
        if end_n == 0:
            return return_trajec, iffull, ifexsitobs

    elif end_n[0].shape[0] == 0 and start_n[0].shape[0] != 0:
        start_n = start_n[0][0]
        end_n = trajectory.shape[0]

    elif end_n[0].shape[0] == 0 and start_n[0].shape[0] == 0:
        start_n = 0
        end_n = trajectory.shape[0]

    else:
        end_n = end_n[0][0]
        start_n = start_n[0][0]

    candidate_seq = trajectory[start_n:end_n]
    offset_start = int((candidate_seq[0, 0] - startframe) // skip)

    offset_end = self.args.seq_length + int((candidate_seq[-1, 0] - endframe) // skip)

    return_trajec[offset_start:offset_end + 1, :3] = candidate_seq

    if return_trajec[self.args.obs_length - 1, 1] != 0:
        ifexsitobs = True

    if offset_end - offset_start >= seq_length - 1:
        iffull = True

    return return_trajec, iffull, ifexsitobs

In [32]:
def massup_batch(batch_data):
    '''
    Massed up data fragements in different time window together to a batch
    '''
    num_Peds = 0
    for batch in batch_data:
        num_Peds += batch.shape[1]

    seq_list_b = np.zeros((self.args.seq_length, 0))
    nodes_batch_b = np.zeros((self.args.seq_length, 0, 2))
    nei_list_b = np.zeros((self.args.seq_length, num_Peds, num_Peds))
    nei_num_b = np.zeros((self.args.seq_length, num_Peds))
    num_Ped_h = 0
    batch_pednum = []
    for batch in batch_data:
        num_Ped = batch.shape[1]
        seq_list, nei_list, nei_num = get_social_inputs_numpy(batch)
        nodes_batch_b = np.append(nodes_batch_b, batch, 1)
        seq_list_b = np.append(seq_list_b, seq_list, 1)
        nei_list_b[:, num_Ped_h:num_Ped_h + num_Ped, num_Ped_h:num_Ped_h + num_Ped] = nei_list
        nei_num_b[:, num_Ped_h:num_Ped_h + num_Ped] = nei_num
        batch_pednum.append(num_Ped)
        num_Ped_h += num_Ped
    return (nodes_batch_b, seq_list_b, nei_list_b, nei_num_b, batch_pednum)

In [33]:
def get_social_inputs_numpy(inputnodes):
    '''
    Get the sequence list (denoting where data exsist) and neighboring list (denoting where neighbors exsist).
    '''
    num_Peds = inputnodes.shape[1]

    seq_list = np.zeros((inputnodes.shape[0], num_Peds))
    # denote where data not missing

    for pedi in range(num_Peds):
        seq = inputnodes[:, pedi]
        seq_list[seq[:, 0] != 0, pedi] = 1

    # get relative cords, neighbor id list
    nei_list = np.zeros((inputnodes.shape[0], num_Peds, num_Peds))
    nei_num = np.zeros((inputnodes.shape[0], num_Peds))

    # nei_list[f,i,j] denote if j is i's neighbors in frame f
    for pedi in range(num_Peds):
        nei_list[:, pedi, :] = seq_list
        nei_list[:, pedi, pedi] = 0  # person i is not the neighbor of itself
        nei_num[:, pedi] = np.sum(nei_list[:, pedi, :], 1)
        seqi = inputnodes[:, pedi]
        for pedj in range(num_Peds):
            seqj = inputnodes[:, pedj]
            select = (seq_list[:, pedi] > 0) & (seq_list[:, pedj] > 0)

            relative_cord = seqi[select, :2] - seqj[select, :2]

            # invalid data index
            select_dist = (abs(relative_cord[:, 0]) > self.args.neighbor_thred) | (
                    abs(relative_cord[:, 1]) > self.args.neighbor_thred)

            nei_num[select, pedi] -= select_dist

            select[select == True] = select_dist
            nei_list[select, pedi, pedj] = 0
    return seq_list, nei_list, nei_num

In [38]:
def get_seq_from_index_balance(self, frameped_dict, pedtraject_dict, data_index, setname):
    """
    Query the trajectories fragments from data sampling index.
    Notes: Divide the scene if there are too many people; accumulate the
    scene if there are few people. This function takes less gpu memory.

    Parameters
    ----------
    frameped_dict : list
        list of dictionaries (1 per scene), linking frame to present
        pedestrians
    pedtraject_dict : list
        list of dictionaries (1 per scene), linking pedestrain to
        trajectories
    data_index : np.array
        np.array containing frames_id and corresponding scenes
    setname : str
        "train" or "test"

    Returns
    -------
    list
        batch_data_mass
    """
    batch_data_mass = []
    batch_data = []
    Batch_id = []

    temp = self.args.batch_around_ped
    if setname == 'train':
        skip = self.trainskip
    else:
        skip = self.testskip

    ped_cnt = 0
    last_frame = 0

    # loop over frames
    for i in range(data_index.shape[1]):
        if i % 100 == 0:
            print("Processed frames:", i, '/', data_index.shape[1])
        cur_frame, cur_set, _ = data_index[:, i]
        framestart_pedi = set(frameped_dict[cur_set][cur_frame])
        try:
            frameend_pedi = set(frameped_dict[cur_set][cur_frame + self.args.seq_length * skip[cur_set]])
        except:
            continue
        present_pedi = framestart_pedi | frameend_pedi
        # if there are only new pedestrians in the end frame
        if (framestart_pedi & frameend_pedi).__len__() == 0:
            continue
        traject = ()
        IFfull = []
        for ped in present_pedi:
            cur_trajec, iffull, ifexistobs = find_trajectory_fragment(pedtraject_dict[cur_set][ped], cur_frame,
                                                                           self.args.seq_length, skip[cur_set])
            if len(cur_trajec) == 0:
                continue
            if ifexistobs == False:
                # Just ignore trajectories if their data don't exsist at the last obversed time step (easy for data shift)
                continue
            if sum(cur_trajec[:, 0] > 0) < 5:
                # filter trajectories have too few frame data
                continue

            cur_trajec = (cur_trajec[:, 1:].reshape(-1, 1, 2),)
            traject = traject.__add__(cur_trajec)
            IFfull.append(iffull)
        if traject.__len__() < 1:
            continue
        if sum(IFfull) < 1:
            continue
        traject_batch = np.concatenate(traject, 1)
        batch_pednum = sum([i.shape[1] for i in batch_data]) + traject_batch.shape[1]

        cur_pednum = traject_batch.shape[1]
        ped_cnt += cur_pednum
        batch_id = (cur_set, cur_frame,)

        if cur_pednum >= self.args.batch_around_ped * 2:
            # too many people in current scene
            # split the scene into two batches
            ind = traject_batch[self.args.obs_length - 1].argsort(0)
            cur_batch_data, cur_Batch_id = [], []
            Seq_batchs = [traject_batch[:, ind[:cur_pednum // 2, 0]], traject_batch[:, ind[cur_pednum // 2:, 0]]]
            for sb in Seq_batchs:
                cur_batch_data.append(sb)
                cur_Batch_id.append(batch_id)
                cur_batch_data = massup_batch(cur_batch_data)
                batch_data_mass.append((cur_batch_data, cur_Batch_id,))
                cur_batch_data = []
                cur_Batch_id = []

            last_frame = i
        elif cur_pednum >= self.args.batch_around_ped:
            # good pedestrian numbers
            cur_batch_data, cur_Batch_id = [], []
            cur_batch_data.append(traject_batch)
            cur_Batch_id.append(batch_id)
            cur_batch_data = massup_batch(cur_batch_data)
            batch_data_mass.append((cur_batch_data, cur_Batch_id,))

            last_frame = i
        else:  # less pedestrian numbers <64
            # accumulate multiple framedata into a batch
            if batch_pednum > self.args.batch_around_ped:
                # enough people in the scene
                batch_data.append(traject_batch)
                Batch_id.append(batch_id)

                batch_data = massup_batch(batch_data)
                batch_data_mass.append((batch_data, Batch_id,))

                last_frame = i
                batch_data = []
                Batch_id = []
            else:
                batch_data.append(traject_batch)
                Batch_id.append(batch_id)

    if last_frame < data_index.shape[1] - 1 and setname == 'test' and batch_pednum > 1:
        batch_data = massup_batch(batch_data)
        batch_data_mass.append((batch_data, Batch_id,))
    self.args.batch_around_ped = temp
    return batch_data_mass

# Execution

In [40]:
frameped_dict, pedtraject_dict = load_dict(data_file)

In [41]:
data_index = get_data_index(self, frameped_dict, setname, ifshuffle=False)

In [42]:
get_seq_from_index_balance(self, frameped_dict, pedtraject_dict, data_index, setname)

Processed frames: 0 / 5528
Processed frames: 100 / 5528
Processed frames: 200 / 5528
Processed frames: 300 / 5528
Processed frames: 400 / 5528
Processed frames: 500 / 5528
Processed frames: 600 / 5528
Processed frames: 700 / 5528
Processed frames: 800 / 5528
Processed frames: 900 / 5528
Processed frames: 1000 / 5528
Processed frames: 1100 / 5528
Processed frames: 1200 / 5528
Processed frames: 1300 / 5528
Processed frames: 1400 / 5528
Processed frames: 1500 / 5528
Processed frames: 1600 / 5528
Processed frames: 1700 / 5528
Processed frames: 1800 / 5528
Processed frames: 1900 / 5528
Processed frames: 2000 / 5528
Processed frames: 2100 / 5528
Processed frames: 2200 / 5528
Processed frames: 2300 / 5528
Processed frames: 2400 / 5528
Processed frames: 2500 / 5528
Processed frames: 2600 / 5528
Processed frames: 2700 / 5528
Processed frames: 2800 / 5528
Processed frames: 2900 / 5528
Processed frames: 3000 / 5528
Processed frames: 3100 / 5528
Processed frames: 3200 / 5528
Processed frames: 3300

[((array([[[ 0.92758 , -1.5855  ],
           [ 1.3182  , -1.7211  ],
           [-0.75585 , -1.4477  ],
           ...,
           [ 0.      ,  0.      ],
           [ 0.      ,  0.      ],
           [ 0.      ,  0.      ]],
   
          [[ 0.92758 , -1.5855  ],
           [ 1.3182  , -1.7211  ],
           [-0.75585 , -1.4477  ],
           ...,
           [ 0.      ,  0.      ],
           [ 0.      ,  0.      ],
           [ 0.      ,  0.      ]],
   
          [[ 0.92758 , -1.5855  ],
           [ 1.3182  , -1.7211  ],
           [-0.75585 , -1.4477  ],
           ...,
           [ 0.      ,  0.      ],
           [ 0.      ,  0.      ],
           [ 0.      ,  0.      ]],
   
          ...,
   
          [[ 0.92758 , -1.5855  ],
           [ 1.3151  , -1.7014  ],
           [-0.64598 , -1.5182  ],
           ...,
           [ 1.3311  ,  1.0681  ],
           [-1.3304  ,  2.6636  ],
           [-0.24795 ,  3.3395  ]],
   
          [[ 0.92758 , -1.5855  ],
           [ 1.3151  ,